In [1]:
!pip install tensorflow-gpu==2.0.0-beta0 #텐서플로우 가장 최신버젼 ( 2019-07-04 )

     |████████████████████████████████| 348.9MB 47kB/s 
     |████████████████████████████████| 3.1MB 40.9MB/s 
     |████████████████████████████████| 501kB 45.9MB/s 


In [2]:
import tensorflow as tf #버젼 확인
tf.__version__

'2.0.0-beta0'

In [3]:
!nvidia-smi # 리눅스 명령어 " GPU 할당 확인"

Thu Jul  4 01:32:14 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8    25W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [5]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [10]:
!ls '/gdrive/My Drive/Colab Notebooks/datas/keras'

2BA90A35.txt  2BA91A46.txt  2BA93A22.txt  2BA93B13.txt	2BB9313.txt
2BA90A44.txt  2BA91A56.txt  2BA93A24.txt  2BA95L58.txt	2BB9314.txt
2BA91A05.txt  2BA92A37.txt  2BA93A32.txt  2BB9301.txt	2BH9910.txt
2BA91A15.txt  2BA92A47.txt  2BA93A34.txt  2BB9304.txt	2BH9911.txt
2BA91A36.txt  2BA93A20.txt  2BA93A48.txt  2BB9306.txt


In [0]:
import os
import re
import numpy as np
import tensorflow as tf

from random import shuffle
from glob import glob
from tqdm import tqdm
from bs4 import BeautifulSoup
from itertools import chain
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint

In [0]:
# Define valid character set
 ##글자 하나하나에 인덱스 부여
  ##유니코드 이용, 쓸 한글들을 define.
complete_kor_range = range(0xac00, 0xd7a4)
  ##유니코드 이용, 쓸 영어들을 define.
complete_eng_range = range(0x0041, 0x007b)
#punctuations = '~!@#$%^&*()-_=+{}[];:\'"<>,./?'
  ##숫자 하나하나에도 인덱스로 부여. 문자로 생각. 숫자가 8이면 붙고 9면 뗄 수도 있다라는 뜻.
numbers = '0123456789'

#mapping 테이블 생성. 인덱스가 들어오면 그 캐릭터가 무엇인지.
idx_to_chr = []
##캐릭터가 들어오면 그 인덱스가 무엇인지 하는 dictionary
chr_to_idx = {}
# The first few indices are reserved
## 0번째 인덱스는 패딩. ( 글자가 부족할 때 남는 값을 무엇으로 채울거냐?) 글자가 없으면 0으로 채우겠다.
idx_to_chr.append('<PAD>')
idx_to_chr.append('<START>')
## 다른 나머지 모르는건 unknown 토큰으로 지정. 위에 지정하지 않은 값들은 모르는 값으로 지정.
idx_to_chr.append('<UNK>') # unknown

# A list mapping indices to words
idx_to_chr.extend(numbers)
#idx_to_chr.extend(punctuations)
for i in chain(complete_kor_range, complete_eng_range):
    idx_to_chr.append(chr(i))

# A dictionary mapping charactes to integer indices
for v, k in enumerate(idx_to_chr):
    chr_to_idx[k] = v

In [16]:
idx_to_chr[0:10]
#인덱스 순서대로 캐릭터가 나옴.


['<PAD>', '<START>', '<UNK>', '0', '1', '2', '3', '4', '5', '6']

In [15]:
chr_to_idx['가']
#'가'라는 글자의 인덱스를 확인.

13

In [0]:
def encode_string(s):
    encoded = []
    for c in s:
        try:
            idx = chr_to_idx[c]
        except:
            idx = chr_to_idx['<UNK>']
        encoded.append(idx)
    return encoded


def get_label(s, idx=0):
    label = []
    while True:
        try:
            next_ch = s[idx + 1]
        except:
            # End of sentence
            label.append(0)
            break
        if next_ch == ' ':
            label.append(1)
            idx += 2
        else:
            label.append(0)
            idx += 1
    return label

In [0]:
##텍스트 파일 하나에 2GB 넣고. 한번에 돌리기에는 컴퓨터에 무리가 생김.
##작은 텍스트 파일 여러개로 불러옴.
def build_dataset(filenames, batch_size=64, max_text_len=200):
    X_batch = []
    y_batch = []
    
    for p in tqdm(filenames):
        # Compiled regex to filter tags
        treg = re.compile('<\w*>|</\w*>')
        # Compiled regex to filter whitespaces
        wreg = re.compile(' ')
        with open(p, 'r', encoding='utf-8') as f:
            # Parse html file
            html = f.readlines()
            html = ''.join(html)
            soup = BeautifulSoup(html, 'html.parser')
            # Use paragraphs only
            texts = soup.find_all('p')
            texts = [treg.sub('', str(t)).strip() for t in texts]
            # Filter zero length input sentences
            # and convert to lowercase
            texts = [t.lower() for t in texts if t]

            # Compute labels
            ## get_label : 문장을 가져왔을 때, 그 글자의 다음 칸에 띄어쓰기가 있으면 1, 없으면 0.
            labels = [get_label(t) for t in texts]
            # Remove whitespaces
            texts = [wreg.sub('', str(t)) for t in texts]
            texts = [encode_string(t) for t in texts]

            X_batch += texts
            y_batch += labels
    
    X_batch = pad_sequences(X_batch, maxlen=max_text_len, 
                            padding='post', truncating='post',
                            value=chr_to_idx['<PAD>'])
    y_batch = pad_sequences(y_batch, maxlen=max_text_len, 
                            padding='post', truncating='post',
                            value=chr_to_idx['<PAD>'])
    
    return (X_batch, y_batch)

In [21]:
##데이터 경로
filenames = glob('/gdrive/My Drive/Colab Notebooks/datas/keras/*.txt')
shuffle(filenames)

train_filenames = filenames[:100]
val_filenames = filenames[100:120]
#test_filenames = filenames[2000:]

print('Total {} files.'.format(len(filenames)))
print('Using {} files for training.'.format(len(train_filenames)))
print('Using {} files for validation.'.format(len(val_filenames)))
#print('Using {} files for test.'.format(len(test_filenames)))

Total 150 files.
Using 100 files for training.
Using 20 files for validation.


In [22]:
##텍스트의 길이 지정. 
max_text_len = 200

X_train, y_train = build_dataset(train_filenames, max_text_len=max_text_len)
print('X_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)

X_val, y_val = build_dataset(val_filenames, max_text_len=max_text_len)
print('X_val.shape: ', X_val.shape)
print('y_val.shape: ', y_val.shape)

#X_test, y_test = build_dataset(test_filenames, max_text_len=max_text_len)
#print('X_test.shape: ', X_test.shape)
#print('y_test.shape: ', y_test.shape)

  0%|          | 0/20 [00:00<?, ?it/s]

X_train.shape:  (106681, 200)
y_train.shape:  (106681, 200)


100%|██████████| 20/20 [00:11<00:00,  1.73it/s]


X_val.shape:  (20794, 200)
y_val.shape:  (20794, 200)


In [25]:
x_train[0] ## 0이 없다. : 200글자를 꽉 채워서.

NameError: ignored

In [24]:
y_train[0]

array([0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0], dtype=int32)

In [26]:
##잘 되었는지 확인.
idx = 99
print('Encoded string')
print(X_train[idx])
print('\nEncoded label')
print(y_train[idx])

print('\nWhen converted back to string')
for i, c in enumerate(X_train[idx]):
    print(idx_to_chr[c], end='')
    if y_train[idx][i] == 1:
        print(' ', end='')

Encoded string
[ 5445   209 10345  7437  6649   409  6989  3765  2141    11    11    41
  7013  5688  7069  6733   573 10821  3177   377  5438  2089  6497 10821
  6621  6649 10646    42  7013  7041  3473  5438  4305  5354  7049   378
  7182  2302  6993   573  7097 10597  2001  3178 10597   237  7061  1777
     2 10629  2337  6481  7041  1697    11    11    41   525  7069  3177
   377  5438  2089  6497 10821  3465  1781     5  8469  3545  6621  7050
  6593  1221  1777     2  5869  6838  8833  2281  3465  7293 10905 10597
  2300  6649   409  6993  7069  2022  7045  5866  7090  8805  6621  9666
   265  5865  9393  3713  8961 10485  9533  3765  1749  9533  6621  7045
  5866  2089  5688  7069  6733  7045  7182  5305 10618  7041 10345  5865
  2089  1777     2  1889  7069  5445 10601  1217  6838  6993  7293  5305
 10597  3121  3713   258 10618 10821  5809  9925  8961 10485  9533  6621
  6653   189  2089  7069  3289  3465  8267  6481 11216  7293 10905     2
  2957  6985  3177  2289  6593    17

In [0]:
##딥러닝이 항상 깊게 가는것은 아니다. 하나의 인풋이 여러개로 들어감 : wide하게 간다.
def conv_block(x, filter_nums, filter_sizes):
    """A convolution block"""
    conv_blocks = []
##필터 넘버와 사이즈를 받음.
    for fn, fs in zip(filter_nums, filter_sizes):
        conv = layers.Conv1D(filters=fn,
                             kernel_size=fs,
                             padding='same',
                             activation='relu',
                             strides=1)(x)
        #conv = layers.GlobalMaxPooling1D()(conv)
        conv_blocks.append(conv)

    return conv_blocks

In [28]:
USE_MULTI_GPU = True ##COLAB에서는 FALSE해도댐

def Model():
    inputs = layers.Input(shape=(max_text_len,)) ##max 200이므로 200개만 가져오겠다.
    ##임베딩을 하겠다 : 아래 코드 추가.
    ##200글자를 가져오고
    ##100 : 100차원으로 맵핑. - 하나의 캐릭터가 100차원으로 구현. - 모델의 수정해보겠다 : 이 값을 바꿔봐라.
    ##input_length 수정 : 들여올 단어 개수를 수정
    ##최종적으로 200 * 100 행렬
    x = layers.Embedding(len(idx_to_chr), 100, input_length=max_text_len)(inputs)
    ##256개에 4그램. 256개에 6그램 ```
    blocks = conv_block(x, filter_nums=(256, 256, 256, 256, 256), 
                           filter_sizes=(4, 6, 8, 10, 12))
    x = layers.Concatenate()(blocks)
    ##LSTM 두번 거침.
    x = layers.LSTM(100, 
                    dropout=0.3,
                    #recurrent_dropout=0.3,
                    return_sequences=True)(x)
    x = layers.LSTM(50,
                    dropout=0.1,
                    #recurrent_dropout=0.1,
                    return_sequences=True)(x)
    ##TD : Dense 값을 (똑같의 오퍼레이션을) 각각의 하나의 엘리멘테이션에 적용해주겠다. : 사이즈(크기를) 조절할 수 있음.
    x = layers.TimeDistributed(layers.Dense(300, activation='relu'))(x)
    x = layers.Dropout(0.3)(x)
    x = layers.TimeDistributed(layers.Dense(150, activation='relu'))(x)
    x = layers.TimeDistributed(layers.Dense(1, activation='sigmoid'))(x)
    ##마지막 라벨이 200차원으로 맞춰짐.
    x = layers.Reshape((200,))(x)

    model = models.Model(inputs, x)

    return model

if USE_MULTI_GPU:
    mirrored_strategy = tf.distribute.MirroredStrategy()
    #mirrored_strategy = tf.distribute.get_strategy()
    with mirrored_strategy.scope():
        model = Model()
        ## loss optimizer 쓰기 편함 그냥 적기만 하면 됨.
        ## accuracy를 많이 쓰면 너무 느려짐. 아래 주석을 하나씩 추가하면 더 알 수 있음.
        model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])
                               #tf.metrics.Precision(),
                               #tf.metrics.Recall(),
                               #tf.metrics.AUC()])

        model.summary()
else:
    model = Model()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
                           #tf.metrics.Precision(),
                           #tf.metrics.Recall(),
                           #tf.metrics.AUC()])
    model.summary()
    
    ##최종적으로 200차원의 값으로 나왔다.

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 200, 100)     1124300     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 200, 256)     102656      embedding[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 200, 256)     153856      embedding[0][0]                  
______________________________________________________________________________________________

In [29]:
MODEL_SAVE_DIR_PATH = './models'
if not os.path.exists(MODEL_SAVE_DIR_PATH):
    os.makedir(MODEL_SAVE_DIR_PATH)

model_path = os.path.join(MODEL_SAVE_DIR_PATH, '{epoch:02d}-{val_loss:.4f}-{val_accuracy:.4f}.hdf5')

checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', verbose=1,
                             save_best_only=True)

AttributeError: ignored

In [0]:
# Compute global batch size using number of replicas.
BATCH_SIZE_PER_REPLICA = 64
global_batch_size = (BATCH_SIZE_PER_REPLICA *
                     mirrored_strategy.num_replicas_in_sync)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(1024)
train_dataset = train_dataset.batch(global_batch_size)
train_dataset = train_dataset.repeat()

valid_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
valid_dataset = valid_dataset.batch(global_batch_size)
valid_dataset = valid_dataset.repeat()

In [0]:
istory = model.fit(train_dataset,
                    epochs=3,
                    validation_data=valid_dataset,
                    steps_per_epoch=len(y_train)//global_batch_size,
                    validation_steps=len(y_val)//global_batch_size,
                    callbacks=[checkpoint],
                    verbose=1)

In [0]:
history = model.fit(X_train, y_train,
                    epochs=3,
                    batch_size=1024,
                    validation_data=(X_val, y_val),
                    callbacks=[checkpoint],
                    verbose=1)

In [0]:
history_dict = history.history
history_dict.keys()

In [0]:
import matplotlib.pyplot as plt

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot([1, 2, 3], [0.0331, 0.0256, 0.0223], 'bx-', label='Training loss')
# b is for "solid blue line"
plt.plot([1, 2, 3], [0.1639, 0.0596, 0.0529], 'ro-', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()